# Scripts for learning 

In [2]:
dimensions_ingredients = 702
dimensions_nutrition_facts = 22
number_of_clusters=100

In [3]:
# Some Imports
from sklearn.cluster import KMeans
import sklearn.metrics as sm
import numpy as np
import json

In [4]:
# Define Inputs
X = None
Y = None

# Load Recipes
filename="recipes.json"
with open(filename) as json_data:
    recipes = json.load(json_data)
    Y = list(recipes.keys())
    X = np.zeros([len(Y), dimensions_ingredients+dimensions_nutrition_facts])
    index = 0
    for label, recipe in recipes.items():
        for ingredient in recipe["ingredients"]:
            X[index, ingredient[0]] = ingredient[1]
        for fact in recipe["nutrition"]:
            X[ index, dimensions_ingredients + fact[0] ] = fact[1]
        index+=1
print("File "+filename+" was successfully read")

Reading file recipes.json


In [5]:
# See one example
print(Y[0])
print(X[0,:])

Boiled Peanuts
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00  

In [6]:
# Train K-Means
# model = KMeans(n_clusters=number_of_clusters)
# model.fit(X)

In [7]:
# Show the labels
# model.labels_

In [9]:
# Show one of the groups
groups = []
for group in range(0, number_of_clusters):
    groups.append([])
#for index in range(0,len(Y)):
    # groups[model.labels_[index]].append(Y[index])

# Now go with nearest neighbors

In [11]:
X_ingredients = X[:,0:dimensions_ingredients]

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X_ingredients)

In [20]:
# Find the nearest neighbor to a given one
def _neighbors_of(X):
    X_=np.mean(X, axis=0)
    X_ = X_.reshape(1,-1)
    neighbors=[]
    distances, indices = nbrs.kneighbors(X_)
    for n_index in indices[0]:
        neighbors.append(Y[int(n_index)])
    return neighbors
    
def neighbors_of(index):
    X=X_ingredients[index:index+1,:]
    return _neighbors_of(X)
    
neighbors_of(100)

['Italian Boiled Pretzels: Taralli',
 'Olive Oil And Seed Crackers',
 'Egg Noodle',
 'Jam or Nutella Filled Doughnuts',
 'Cinnamon Bread',
 'Bacon, Walnut And Thyme Scones',
 'Panettone',
 'Strawberries And Cream Melting Moments',
 'Fried Apple Pies',
 'Sugared Cream Scones']

In [19]:
# Or find the nearest to a group of those
def neighbors_of_list(elements, cluster=False):
    X=X_ingredients[elements,:]
    if cluster:
        print("Not implemented")
    else:
        return _neighbors_of(X)
        
neighbors_of_list([0, 268, 3930, 100])

['Vietnamese Peanut Sauce',
 "Fast Breads' Crusty Artisanal Bread",
 'Menudo',
 'Tomato Lemongrass Soup',
 'Dinner Tonight: Tunisian Chickpea Soup',
 'California Rolls',
 'Cook the Book: Dashi and Japanese Chicken Stock',
 'Hearty Vietnamese Beef Noodle Soup (Pho Bo)',
 'Earl Grey Tea Mooncake with Egg Yolk and Pine Nuts',
 'Cherry & Thai Basil Soda']